<a href="https://colab.research.google.com/github/djmayo04/AI_Mini-Project2/blob/main/notebooks/AI_MP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-groq groq

  Using cached groq-1.0.0-py3-none-any.whl.metadata (16 kB)


In [2]:
import os

REPO_URL = "https://github.com/djmayo04/AI_Mini-Project2.git"
REPO_DIR = "AI_Mini-Project2"

%cd /content

if not os.path.exists(REPO_DIR):
    !git clone "{https://github.com/djmayo04/AI_Mini-Project2.git}"
else:
    %cd {REPO_DIR}
    !git pull
    %cd /content

%cd {REPO_DIR}
!ls

/content
/content/AI_Mini-Project2
Already up to date.
/content
/content/AI_Mini-Project2
agent.py       memory.py  __pycache__  search_documents.py
llm_helper.py  notebooks  README.md    tools.py


In [3]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
print(os.environ.get("GROQ_API_KEY") is not None)

True


In [4]:
from llm_helper import get_completion
from agent import run_agent

print(get_completion("Say hello in one sentence."))
print(run_agent("What does the syllabus say about deadlines?"))

Hello!
I’m sorry—I don’t have the specific syllabus text you’re referring to, so I can’t quote its exact policy on deadlines. If you can share the relevant portion of the syllabus (or let me know where it’s posted), I’ll be happy to pull out the details for you.


In [7]:
!ls notebooks

04_groq.ipynb
